In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = 'data'

In [ ]:
df_demograph = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/demographic_cleaned.csv'), dtype={'zip3': str})

nevada_zip = [str(x) for x in range(850, 950)]
nyc_zip = [str(x) for x in range(100, 110)]

zip_fil = nyc_zip
mask = df_demograph['zip3'].isin(nyc_zip)
df_demograph = df_demograph[mask]

In [ ]:
df_acp = pd.read_pickle(os.path.join(DATA_DIR, 'accepted_merged.pickle'))

In [ ]:
race = 'black'
normalize_period = '2017Q4'
top_n = 30

black_zip = df_demograph.groupby('zip3')[f'is_{race}'].mean()
black_zip = black_zip.sort_values(ascending=False).index[:top_n].tolist()

# black_zip = df_acp.groupby('zip3')[f'is_{race}'].first().sort_values(ascending=False).index[:top_n].tolist()

loan_df = df_acp[['funded_amnt', 'issue_d', 'zip3']]
loan_df['issue_q'] = pd.to_datetime(loan_df['issue_d']).dt.to_period('Q')

loan_black = loan_df[loan_df['zip3'].isin(black_zip)]
loan_non_black = loan_df[~loan_df['zip3'].isin(black_zip)]

loan_black_amt = loan_black.groupby('issue_q')['funded_amnt'].sum()
loan_black_amt /= loan_black_amt[normalize_period]
loan_non_black_amt = loan_non_black.groupby('issue_q')['funded_amnt'].sum()
loan_non_black_amt /= loan_non_black_amt[normalize_period]

fig = plt.figure(figsize=(8,6))
loan_black_amt.plot(label=f'{race}')
loan_non_black_amt.plot(label=f'non-{race}')
plt.axvline(x=normalize_period, color = 'black',  linestyle='--')
plt.title(f'Loan Amount around LC Scandal')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Normalized Average Funded Amout')
# plt.show()
plt.savefig('fund_amnt_{0}_{1}.png'.format(race, normalize_period))

In [ ]:
len(loan_black), len(loan_non_black)